### Predicting If School is Public or Private

### Decision Tree (DT) vs Random Forest (RF) vs Gradient Boosted (GBT)

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('private_vs_public').master('local[4]').getOrCreate()

### Loading Data

In [3]:
data = spark.read.csv('../data/College.csv', inferSchema=True, header=True)

In [9]:
data.count()

777

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [14]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

### Assemble Features

In [13]:
from pyspark.ml.feature import VectorAssembler

In [15]:
inputCols = [
    'Apps',
    'Accept',
    'Enroll',
     'Top10perc',
     'Top25perc',
     'F_Undergrad',
     'P_Undergrad',
     'Outstate',
     'Room_Board',
     'Books',
     'Personal',
     'PhD',
     'Terminal',
     'S_F_Ratio',
     'perc_alumni',
     'Expend',
     'Grad_Rate']

In [16]:
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [19]:
output = assembler.transform(data)

In [21]:
output.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate',
 'features']

### Encode the Target Attribute

In [22]:
from pyspark.ml.feature import StringIndexer

In [23]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [24]:
output_fixed = indexer.fit(output).transform(output)

In [25]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [30]:
final_data = output_fixed.select('features', 'PrivateIndex')

### Train Test Split

In [31]:
train_data, test_data = final_data.randomSplit([0.8, 0.2])

### Train Models

In [33]:
from pyspark.ml.classification import (
    DecisionTreeClassifier,
    RandomForestClassifier,
    GBTClassifier)

In [34]:
from pyspark.ml import Pipeline

In [36]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [37]:
dtc_model = dtc.fit(train_data)
rf_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [38]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rf_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

### Evaluation

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [51]:
print('DTC AUC', bin_eval.evaluate(dtc_preds))
print('RF AUC', bin_eval.evaluate(rfc_preds))
print('GBT AUC', bin_eval.evaluate(gbt_preds))

DTC AUC 0.9105646630236793
RF AUC 0.9814207650273222
GBT AUC 0.9674863387978149


In [46]:
# BUT WAIT, GBT Predictions don't have the Raw Predictions Column
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [53]:
# This can cause errors when using BinaryClassificationEvaluator
# We need to define the name of rawPredictionCol to resolve this

In [54]:
bin_eval_2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [55]:
print('GBT AUC', bin_eval_2.evaluate(gbt_preds))

GBT AUC 0.9017304189435337


In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [57]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',
                                            metricName='accuracy')

In [58]:
acc_eval.evaluate(gbt_preds)

0.9281437125748503